In [ ]:
# # Importe as bibliotecas necessárias
# import pandas as pd
# import holidays
# import plotly.express as px
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# from pycaret.time_series import *

In [ ]:
# # Carregue os dados
# data = pd.read_csv('../TCC/datasets/demanda.csv')
# data['Timestamp'] = pd.to_datetime(data['Timestamp'])
# data = data.set_index('Timestamp').asfreq('H')

# # Defina a coluna alvo
# target = 'kVA fornecido'

# # Trabalhando com a coluna alvo
# data = data[[target]]

# # Criando colunas de numéricas
# data['Hour'] = data.index.hour
# data['Day'] = data.index.day
# data['Dayofweek'] = data.index.dayofweek
# data['Month'] = data.index.month
# data['Quarter'] = data.index.quarter
# data['Year'] = data.index.year
# br_holidays = holidays.Brazil(years=data.index.year.unique())
# data['Holiday'] = data.index.map(lambda x: x in br_holidays).astype(int)

# # Lista de colunas numéricas
# num_cols = data.columns.drop(target).tolist()

# # Movendo os últimos 24 dados (1 dia) para serem usados como dados de validação da previsão
# df_forecast = data.iloc[-24:]
# data = data.iloc[:-24]

# # Separando os dados de treino e teste
# slice_point = '2023-02-01'
# train = data[data.index < slice_point]
# test = data[data.index  >= slice_point]

# Pycaret

In [ ]:
# # Inicializando o ambiente
# s = setup(data = data, fh=24, session_id=42, log_experiment=True, target=target, experiment_name='kVA fornecido')

In [ ]:
# # import TSForecastingExperiment and init the class
# from pycaret.time_series import TSForecastingExperiment
# exp = TSForecastingExperiment()

In [ ]:
# # init setup on exp
# exp.setup(data = data, fh=24, session_id=42, log_experiment=True, target=target, experiment_name='kVA fornecido')

In [ ]:
# # check statistical tests on original data
# check_stats()

In [ ]:
# # compare baseline models
# best = compare_models()

In [ ]:
# # compare models using OOP
# exp.compare_models()

In [ ]:
# # plot forecast
# plot_model(best, plot = 'forecast')

In [ ]:
# # plot forecast for 24 hours in future
# plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 24})

In [ ]:
# # residuals plot
# plot_model(best, plot = 'residuals')

In [ ]:
# # predict on test set
# holdout_pred = predict_model(best)

In [ ]:
# # generate forecast for 24 period in future
# predict_model(best, fh = 24)

# Pycaret 2

In [ ]:
# Importe as bibliotecas necessárias
import pandas as pd
import holidays
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pycaret.time_series import *

# Carregue os dados
data = pd.read_csv('../TCC/datasets/demanda.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data = data.set_index('Timestamp').asfreq('H')

# Defina a coluna alvo
target = 'kVA fornecido'

# Trabalhando com a coluna alvo
data = data[[target]]

# Criando colunas de numéricas
data['Hour'] = data.index.hour
data['Day'] = data.index.day
data['Dayofweek'] = data.index.dayofweek
data['Month'] = data.index.month
data['Quarter'] = data.index.quarter
data['Year'] = data.index.year
br_holidays = holidays.Brazil(years=data.index.year.unique())
data['Holiday'] = data.index.map(lambda x: x in br_holidays).astype(int)

# Lista de colunas numéricas
num_cols = data.columns.drop(target).tolist()

# Movendo os últimos 24 dados (1 dia) para serem usados como dados de validação da previsão
df_forecast = data.iloc[-24:]
data = data.iloc[:-24]

In [94]:
# Inicialize o ambiente do PyCaret
exp = setup(data, 
            fh=24,
            target=target,
            session_id=123, 
            fold_strategy='expanding', 
            fold=5,
            seasonal_period=24,
            seasonality_type='auto',
            )

,Description,Value
0,session_id,123
1,Target,kVA fornecido
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(10153, 8)"
5,Transformed data shape,"(10153, 8)"
6,Transformed train set shape,"(10129, 8)"
7,Transformed test set shape,"(24, 8)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [95]:
# Compare modelos
best = compare_models(fold=5, sort='RMSE', n_select=1)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.4217,0.3388,256.5366,327.0801,0.4091,0.1972,-0.3641,3.8480
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.5087,0.4196,309.4788,405.1298,0.5194,0.2363,-1.3508,0.6260
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.5786,0.4599,351.9221,443.9774,0.4625,0.2441,-1.4854,3.5780
croston,Croston,0.5436,0.4644,330.4543,448.0281,0.9300,0.2380,-0.4829,0.0420
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.5695,0.4644,346.5210,448.4042,0.4548,0.2387,-2.1685,7.2320
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.5696,0.4644,346.5524,448.4447,0.4548,0.2387,-2.1688,0.4200
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,0.5811,0.4683,353.4749,452.0410,0.4558,0.2390,-1.4585,1.7180
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.5743,0.4718,349.3958,455.5068,0.5059,0.2472,-2.0543,1.0520
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,0.5554,0.4763,337.9390,459.9812,0.4373,0.2638,-3.8014,0.3960
arima,ARIMA,0.5997,0.4854,364.7746,468.5643,0.5866,0.2689,-1.9520,25.5940


In [96]:
# Tunando o modelo
tuned_best = tune_model(best, fold=5, optimize='RMSE', choose_better=True)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2023-02-22 00:00,0.5552,0.4859,338.3742,469.8710,0.1811,0.2182,-5.5624
1,2023-02-23 00:00,0.5937,0.4242,361.5106,409.8935,0.1995,0.2026,-0.1528
2,2023-02-24 00:00,0.8234,0.6177,500.7262,596.2277,0.2982,0.3551,-2.0383
3,2023-02-25 00:00,0.5902,0.4189,358.5363,403.9753,0.2627,0.2269,-0.1718
4,2023-02-26 00:00,0.4236,0.3248,257.1432,312.9694,0.9216,0.3616,0.7277
Mean,NaT,0.5972,0.4543,363.2581,438.5874,0.3726,0.2729,-1.4395
SD,NaT,0.1289,0.0966,78.4887,93.4134,0.2777,0.0702,2.2502


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.8min finished


In [97]:
# Treine o modelo selecionado
model = finalize_model(tuned_best)

In [98]:
# Gerando dados de previsão para 24 horas no futuro
future_data = df_forecast.drop(target, axis=1)

In [99]:
# Faça previsões para os próximos 24 passos de tempo
future = 24
forecast = predict_model(model, X=future_data, fh=future)

In [100]:
# Insert 'y_pred' column into df_forecast
df_forecast['y_pred'] = forecast['y_pred'].values

In [101]:
#plot using plotly
fig = px.line(df_forecast, x=df_forecast.index, y=[target, 'y_pred'], 
              title='Forecast for next 24 hours')
fig.show()